In [1]:
import sys
sys.path.append("..")

In [2]:
label_path = '../output/label.pkl'
file = '/Users/knpob/Territory/Kolmo/data/DynaFootLite/Fast/Sub05_Fast_L1_102_143.000006.obj'
is_plot = True

## Load landmarks labelling

In [3]:
import pandas as pd

df = pd.read_pickle(label_path)
df

coord                                                                x  \
file                                               landmark              
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1       -47.868675   
                                                   P10       10.347754   
                                                   P11       27.356030   
                                                   P12      -40.068095   
                                                   P2        -6.773722   
                                                   P3       -76.020657   
                                                   P4         7.909824   
                                                   P5       -66.313779   
                                                   P6       -15.675663   
                                                   P7       -11.845389   
                                                   P8        33.788057   
                                                   P9       -24.513464   

coord                                                                 y  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1       -102.082951   
                                                   P10      -141.225175   
                                                   P11       -66.791808   
                                                   P12       -98.773372   
                                                   P2       -102.914672   
                                                   P3       -115.615363   
                                                   P4       -105.273731   
                                                   P5       -133.365319   
                                                   P6        -75.853272   
                                                   P7        -63.623706   
                                                   P8       -143.022407   
                                                   P9       -149.428966   

coord                                                                 z  
file                                               landmark              
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1        -27.146627  
                                                   P10       191.482570  
                                                   P11       141.348713  
                                                   P12       148.841208  
                                                   P2        -29.897389  
                                                   P3         27.144024  
                                                   P4          7.795904  
                                                   P5         50.973984  
                                                   P6         80.303229  
                                                   P7        106.223576  
                                                   P8        163.758241  
                                                   P9        172.440759

## Local frame

In [4]:
import pyvista as pv
from measure import frame, visual
from mesh4d.analyse import crave

mesh = crave.fix_pvmesh_disconnect(pv.read(file))
axes_frame, origin = frame.estimate_foot_frame(mesh, file, df)

if is_plot:
    visual.plot_axes(origin, axes_frame, mesh)

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e27e6a00_0&reconnect=auto' style='widt…

In [5]:
mesh_clip = frame.foot_clip(mesh, df, file)
mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)

if is_plot:
    mesh_local.plot()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84d07990a0_1&reconnect=auto' style='widt…

In [6]:
df_local = frame.df2local(df, axes_frame, origin)
df_local

coord                                                                 x  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1        104.877372   
                                                   P10      -124.412992   
                                                   P11       -64.506712   
                                                   P12       -63.916425   
                                                   P2         98.482753   
                                                   P3         56.299284   
                                                   P4         58.881029   
                                                   P5         27.729230   
                                                   P6          1.043945   
                                                   P7        -21.907594   
                                                   P8       -103.399672   
                                                   P9       -100.491941   

coord                                                                y  \
file                                               landmark              
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1         9.131839   
                                                   P10       -5.143694   
                                                   P11      -38.117774   
                                                   P12       32.423743   
                                                   P2       -31.516304   
                                                   P3        47.691094   
                                                   P4       -38.964888   
                                                   P5        44.186342   
                                                   P6        -5.919685   
                                                   P7        -6.284384   
                                                   P8       -32.843289   
                                                   P9        26.429013   

coord                                                                z  
file                                               landmark             
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1        24.566920  
                                                   P10       33.195440  
                                                   P11       93.100196  
                                                   P12       67.595971  
                                                   P2        20.771757  
                                                   P3        25.448109  
                                                   P4        26.259271  
                                                   P5        13.089730  
                                                   P6        72.796017  
                                                   P7        90.393067  
                                                   P8        23.761532  
                                                   P9        22.865127

## Metrics

In [7]:
results = [
    {
        'file': 'description',
        'FL': 'foot length (mm)',
        'MBL': 'medial ball length (mm)',
        'LBL': 'lateral ball length (mm)',
        'ABW': 'anatomical ball width (mm)',
        'OBW': 'orthogonal ball width (mm)',
        'OHW': 'orthogonal heel width (mm)',
        'BH': 'ball heigh (mm)',
        'IH': 'instep height (mm)',
        'BA': 'ball angle (°)',
        'T1A': 'toe 1 angle (°)',
        'T5A': 'toe 5 angle (°)',
        'ABG': 'anatomical ball girth (mm)',
        'IG': 'instep girth (mm)',
    }
]

In [8]:
from measure import metric

results.append(
    {
        'file': file,
        'FL': metric.fl(df_local, file),
        'MBL': metric.mbl(df_local, file),
        'LBL': metric.lbl(df_local, file),
        'ABW': metric.abw(df_local, file),
        'OBW': metric.obw(df_local, file),
        'OHW': metric.ohw(df_local, file),
        'BH': metric.bh(df_local, file),
        'IH': metric.ih(df_local, file),
        'BA': metric.ba(df_local, file),
        'T1A': metric.t1a(df_local, file),
        'T5A': metric.t5a(df_local, file),
        'ABG': metric.abg(df_local, file, mesh_local),
        'IG': metric.ig(df_local, file, mesh_local),
    }
)

In [9]:
df_metric = pd.DataFrame(results).set_index('file')
df_metric

,FL,MBL,LBL,ABW,OBW,OHW,BH,IH,BA,T1A,T5A,ABG,IG
file,,,,,,,,,,,,,
description,foot length (mm),medial ball length (mm),lateral ball length (mm),anatomical ball width (mm),orthogonal ball width (mm),orthogonal heel width (mm),ball heigh (mm),instep height (mm),ball angle (°),toe 1 angle (°),toe 5 angle (°),anatomical ball girth (mm),instep girth (mm)
/Users/knpob/Territory/Kolmo/data/DynaFootLite/Fast/Sub05_Fast_L1_102_143.000006.obj,229.290364,183.294021,152.142222,89.766355,86.655983,59.272302,72.796017,90.393067,67.423662,164.57814,152.431569,125.266833,151.507285


### Visualization

In [10]:
if is_plot:
    settings = {
        'FL': ['P1', 'P10', 'x'],
        'MBL': ['P4', 'P10', 'x'],
        'LBL': ['P5', 'P10', 'x'],
        'OBW': ['P4', 'P3', 'y'],
        'OHW': ['P9', 'P8', 'y'],
    }

    for name, [landmark1, landmark2, axis] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_dist_along_axis(scene, df_local, file, landmark1, landmark2, axis, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e48b52e0_2&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e4736dc0_3&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e48b6dc0_4&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e4707d90_5&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e4707a90_6&reconnect=auto' style='widt…

In [11]:
if is_plot:
    settings = {
        'BH': 'P6',
        'IH': 'P7',
    }

    for name, landmark in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_height(scene, df_local, file, landmark, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e4be51c0_7&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e4be5850_8&reconnect=auto' style='widt…

In [12]:
if is_plot:
    settings = {
        'BA': ['P4', 'P5', 'P8'],
        'T1A': ['P4', 'P2', 'P8'],
        'T5A': ['P5', 'P3', 'P9'],
    }

    for name, [landmark_origin, landmark1, landmark2] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_angle(scene, df_local, file, landmark_origin, landmark1, landmark2, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84d1ea7d90_9&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e48b6df0_10&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84e4be5dc0_11&reconnect=auto' style='wid…

In [13]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_landmarks(scene, df_local, file, mesh_local, ['P4', 'P5'], name='ABG')
    scene.show()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84d1ea78b0_12&reconnect=auto' style='wid…

In [14]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_landmark(scene, df_local, file, mesh_local, 'P6', 'x', name='IG')
    scene.show()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84c110fd00_13&reconnect=auto' style='wid…